In [1]:
import random
random.seed(0)

import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)

We are using seed() so that every time we execute the code we would have same results

Now,we have to import tensorflow module

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [3]:
# Image dimensions
img_width, img_height = 256, 256
input_shape = (img_width, img_height, 3)
num_classes = 6 #Class indices of Dataset i.e the number of folders our data set consists of.

In [4]:
#Dataset path 
data_dir='C:\\Users\\Bhargavi Nagulapally\\Downloads\\rice-leaf'

In [5]:
# Checking  if directories exist
assert os.path.exists(data_dir), "Dataset directory does not exist"
print("Dataset directory exists")

Dataset directory exists


Generating data for training and validation

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

Now,By using flow_from_directory we extract each data from train_datagen and then we will be training the model 

In [7]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='training')

Found 2106 images belonging to 6 classes.


In [8]:
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 522 images belonging to 6 classes.


Printing the class indices or the number of class the dataset consists of

In [9]:
print("Class indices:", train_generator.class_indices)

Class indices: {'bacterial_leaf_blight': 0, 'brown_spot': 1, 'healthy': 2, 'leaf_blast': 3, 'leaf_scald': 4, 'narrow_brown_spot': 5}


Printing the  number of samples that have been categorised as train and validation samples

In [10]:
print(f"Found {train_generator.samples} training samples")
print(f"Found {validation_generator.samples} validation samples")

Found 2106 training samples
Found 522 validation samples


To ensure that  there are samples in the generators,we use assert to check 

In [11]:
assert train_generator.samples > 0, "No training samples found"
assert validation_generator.samples > 0, "No validation samples found"

Creating a function for cnn model

In [12]:
def create_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Function call for the cnn model

In [13]:
cnn_model = create_cnn_model(input_shape, num_classes)

c:\Users\Bhargavi Nagulapally\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
train_generator.batch_size

32

In [15]:
validation_generator.batch_size

32

In [16]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

In [17]:
print(f"steps_per_epoch: {steps_per_epoch}")
print(f"validation_steps: {validation_steps}")

steps_per_epoch: 65
validation_steps: 16


In [18]:
# Ensure steps_per_epoch and validation_steps are greater than 0
if steps_per_epoch == 0:
    steps_per_epoch = 1
    print("Adjusted steps_per_epoch to 1")

if validation_steps == 0:
    validation_steps = 1
    print("Adjusted validation_steps to 1")

In [21]:
# Train the model
history=cnn_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=35)

Epoch 1/35
65/65 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.6547 - loss: 0.9381 - val_accuracy: 0.6758 - val_loss: 0.8201
Epoch 2/35
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6250 - loss: 1.0112 - val_accuracy: 0.8000 - val_loss: 0.4961
Epoch 3/35
65/65 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.7182 - loss: 0.7576 - val_accuracy: 0.6465 - val_loss: 0.8486
Epoch 4/35
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6875 - loss: 0.7836 - val_accuracy: 0.9000 - val_loss: 0.4114
Epoch 5/35
65/65 ━━━━━━━━━━━━━━━━━━━━ 187s 3s/step - accuracy: 0.7523 - loss: 0.6133 - val_accuracy: 0.6797 - val_loss: 0.7778
Epoch 6/35
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7188 - loss: 0.8749 - val_accuracy: 0.7000 - val_loss: 0.8586
Epoch 7/35
65/65 ━━━━━━━━━━━━━━━━━━━━ 187s 3s/step - accuracy: 0.7730 - loss: 0.5676 - val_accuracy: 0.7695 - val_loss: 0.5996
Epoch 8/35
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8125 - loss: 0.5906 - val_accuracy: 0.7000 - val_l

In [22]:
# Model Evaluation
print("Evaluating model...")
val_loss, val_accuracy = cnn_model.evaluate(validation_generator, steps=validation_generator.samples //validation_generator.batch_size)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Evaluating model...
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 955ms/step - accuracy: 0.8412 - loss: 0.5160
Validation Accuracy: 83.40%


In [33]:
# Function to Load and Preprocess the Image using Pillow
from PIL import Image
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    # Load the image
    img = Image.open(image_path)
    # Resize the image
    img = img.resize(target_size)
    # Convert the image to a numpy array
    img_array = np.array(img)
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    # Scale the image values to [0, 1]
    img_array = img_array.astype('float32') / 255.
    return img_array

# Function to Predict the Class of an Image
def predict_image_class(cnn_model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = cnn_model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name

In [34]:
# Create a mapping from class indices to class names
class_indices = {v: k for k, v in train_generator.class_indices.items()}

In [35]:
class_indices

{0: 'bacterial_leaf_blight',
 1: 'brown_spot',
 2: 'healthy',
 3: 'leaf_blast',
 4: 'leaf_scald',
 5: 'narrow_brown_spot'}

In [36]:
# saving the class names as json file
import json
json.dump(class_indices, open('class_indices.json', 'w'))

In [37]:


image_path = 'C:\\Users\\Bhargavi Nagulapally\\OneDrive\\Desktop\\ugc\\DQN_CNN\\RiceLeafsDisease\\validation\\bacterial_leaf_blight\\bacterial_val (14).JPG'
predicted_class_name = predict_image_class(cnn_model, image_path, class_indices)

# Output the result
print("Predicted Class Name:", predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Predicted Class Name: bacterial_leaf_blight


In [38]:
cnn_model.save('Rice_leaf_disease_detection.h5')